<a href="https://colab.research.google.com/github/CRSpradlin/natural-language-processing-course/blob/main/NLP%20Course%20Work/10.%20Twitter%20Sentiment%20Analysis/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# data link: https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv

In [67]:
import numpy as np
import pandas as pd


In [68]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv')
df

,twitts,sentiment
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0
...,...,...
29995,@Calumfan1 is it in any way related to photosh...,0
29996,@Swiz_NZ really? wow thats crap,0
29997,"At the 2010 lexus HS250h press event. Again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


1's are a positive sentiment and 0's are a negative sentiment.

In [70]:
df['sentiment'].value_counts()

sentiment
1    15000
0    15000
Name: count, dtype: int64

## SVM Model and Data Preparation

In [71]:
tweet_content = df['twitts']
tweet_sentiment = df['sentiment']

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(tweet_content)

x_train, x_test, y_train, y_test = train_test_split(x, tweet_sentiment, test_size = 0.2, random_state = 0, stratify = tweet_sentiment)

print('shape of x: ', x.shape)

clf = LinearSVC()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

shape of x:  (30000, 40854)


In [72]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



In [73]:
def run_svm(df):
  tweet_content = df['twitts']
  tweet_sentiment = df['sentiment']

  tfidf = TfidfVectorizer()
  x = tfidf.fit_transform(tweet_content)

  x_train, x_test, y_train, y_test = train_test_split(x, tweet_sentiment, test_size = 0.2, random_state = 0, stratify = tweet_sentiment)

  print('shape of x: ', x.shape)

  clf = LinearSVC()
  clf.fit(x_train, y_train)

  y_pred = clf.predict(x_test)

  print("\nClassification Report")
  print(classification_report(y_test, y_pred))

  return tfidf, clf

In [74]:
%%time
tfidf, clf = run_svm(df)

shape of x:  (30000, 40854)

Classification Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 815 ms, sys: 3.86 ms, total: 819 ms
Wall time: 820 ms


Now to test the model using the generated clf.

In [75]:
x = 'i am excited to see my best friend to get married'

In [76]:
clf.predict(tfidf.transform([x]))

array([1])

The example has been deemed as a positive sentiment statement.

## Data Cleaning and Retraining SVM
Now it is time to use the python package we created to help perform some data cleaning.

In [77]:
!pip install "git+https://github.com/CRSpradlin/natural-language-processing-course.git#egg=preprocess_crspradlin&subdirectory=NLP Course Work/7. Packaging/Preprocessing Package"

  Cloning https://github.com/CRSpradlin/natural-language-processing-course.git to /tmp/pip-install-wdalzflf/preprocess-crspradlin_e318fd9be9d04e48b4ea28a9f273fcfc
  Running command git clone --filter=blob:none --quiet https://github.com/CRSpradlin/natural-language-processing-course.git /tmp/pip-install-wdalzflf/preprocess-crspradlin_e318fd9be9d04e48b4ea28a9f273fcfc
  Resolved https://github.com/CRSpradlin/natural-language-processing-course.git to commit 74464ea17c49105e49abec4665cc559a3ed3df9b
  Preparing metadata (setup.py) ... done


In [78]:
import preprocess_crspradlin as prep

In [79]:
prep.__version__

'0.0.1'

In [80]:
# Apply lowercase modifier
df['twitts'] = df.twitts.apply(lambda x: x.lower())

In [81]:
# Apply Contraction to Expansion modifier
df['twitts'] = df.twitts.apply(lambda x: prep.get_contraction_to_expansion(x))

In [82]:
run_svm(df)

shape of x:  (30000, 40850)

Classification Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.75      0.76      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



(TfidfVectorizer(), LinearSVC())

Not a major improvement overall but there is little improvement in recall.

In [83]:
# Apply removal modifiers
df['twitts'] = df.twitts.apply(lambda x: prep.remove_emails(x))
df['twitts'] = df.twitts.apply(lambda x: prep.remove_urls(x))
df['twitts'] = df.twitts.apply(lambda x: prep.remove_rt(x))
df['twitts'] = df.twitts.apply(lambda x: prep.remove_html_tags(x))
df['twitts'] = df.twitts.apply(lambda x: prep.remove_special_chars(x))

/usr/local/lib/python3.10/dist-packages/preprocess_crspradlin/utils.py:168: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text().strip()


In [84]:
tfidf, clf = run_svm(df)

shape of x:  (30000, 43375)

Classification Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.74      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000



Still no change in results but the data has been "cleaned up".

In [85]:
clf.predict(tfidf.transform([x]))

array([1])

## Fine Tuning the Model
Trying to get the model to have a higher percentage of accuracy.

In [86]:
def run_svm(df):
  tweet_content = df['twitts']
  tweet_sentiment = df['sentiment']

  ## UPDATES HERE ##
  tfidf = TfidfVectorizer(norm = 'l1')
  ####
  x = tfidf.fit_transform(tweet_content)

  x_train, x_test, y_train, y_test = train_test_split(x, tweet_sentiment, test_size = 0.2, random_state = 0, stratify = tweet_sentiment)

  print('shape of x: ', x.shape)

  clf = LinearSVC()
  clf.fit(x_train, y_train)

  y_pred = clf.predict(x_test)

  print("\nClassification Report")
  print(classification_report(y_test, y_pred))

  return tfidf, clf

In [87]:
run_svm(df)

shape of x:  (30000, 43375)

Classification Report
              precision    recall  f1-score   support

           0       0.76      0.75      0.76      3000
           1       0.75      0.76      0.76      3000

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000



(TfidfVectorizer(norm='l1'), LinearSVC())

With the changes to the TfidfVectorizer, you can see an increase to the results. Lets try to make additional adjustments to see how high the percentages get.

In [88]:
def run_svm(df):
  tweet_content = df['twitts']
  tweet_sentiment = df['sentiment']

  ## UPDATES HERE ##
  tfidf = TfidfVectorizer(norm = 'l1', ngram_range=(1,5), analyzer='word', max_features=5000)
  ####
  x = tfidf.fit_transform(tweet_content)

  x_train, x_test, y_train, y_test = train_test_split(x, tweet_sentiment, test_size = 0.2, random_state = 0, stratify = tweet_sentiment)

  print('shape of x: ', x.shape)

  clf = LinearSVC()
  clf.fit(x_train, y_train)

  y_pred = clf.predict(x_test)

  print("\nClassification Report")
  print(classification_report(y_test, y_pred))

  return tfidf, clf

In [89]:
tfidf, clf = run_svm(df)

shape of x:  (30000, 5000)

Classification Report
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      3000
           1       0.76      0.74      0.75      3000

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000



## Saving and Loading ML Model

In [90]:
import pickle

In [91]:
pickle.dump(clf, open('clf.pkl', 'wb')) # Write obj in write-binary mode
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [92]:
del clf
del tfidf
# print(clf) # fails since clf and tfidf are no longer defined

In [93]:
clf = pickle.load(open('clf.pkl', 'rb')) # open file in read-binary mode

In [94]:
tfidf = pickle.load(open('tfidf.pkl', 'rb'))

In [95]:
clf

LinearSVC()

In [96]:
tfidf.vocabulary_

{'sounds': 3668,
 'like': 2371,
 'great': 1574,
 'night': 2834,
 'sounds like': 3670,
 'damn': 874,
 'the': 3892,
 'person': 3200,
 'who': 4672,
 'my': 2710,
 'may': 2534,
 'come': 784,
 'back': 402,
 'and': 215,
 'you': 4887,
 'in': 1908,
 'ass': 356,
 'come back': 785,
 'back and': 403,
 'you in': 4937,
 'in the': 1939,
 'from': 1383,
 'photo': 3207,
 'from the': 1387,
 'love': 2454,
 'it': 2084,
 'haha': 1612,
 'forget': 1359,
 'should': 3512,
 'give': 1461,
 'me': 2541,
 'lie': 2366,
 'please': 3234,
 'would': 4816,
 'be': 440,
 'awesome': 392,
 'if': 1872,
 'did': 940,
 'love it': 2457,
 'love you': 2467,
 'give me': 1463,
 'it would': 2161,
 'would be': 4817,
 'if you': 1886,
 'you did': 4913,
 'it would be': 2162,
 'pretty': 3268,
 'then': 4025,
 'site': 3549,
 'skin': 3556,
 'really': 3327,
 'upset': 4426,
 'big': 510,
 'pool': 3245,
 'or': 3115,
 'might': 2609,
 'able': 38,
 'to': 4137,
 'but': 588,
 'that': 3842,
 'is': 1973,
 'as': 338,
 'will': 4693,
 'get': 1417,
 'might b

In [97]:
clf.predict(tfidf.transform([x]))

array([1])

## Real-Time Twitter Sentiment Analysis

In [98]:
!pip install tweepy

from google.colab import userdata
# userdata.get('consumer_key') # X developer Client ID
# userdata.get('consumer_secret') # X developer Client Secret
# userdata.get('access_token') # X developer Access Token
# userdata.get('access_token_secret') # X developer Access Secret

In [99]:
import tweepy

#auth = tweepy.OAuth2BearerHandler(userdata.get('bearer_token'))

#auth = tweepy.Client(userdata.get('consumer_key'), userdata.get('consumer_secret'), userdata.get('access_token'), userdata.get('access_token_secret'))


#api = tweepy.API(auth)

#public_tweets = api.home_timeline()

#client = tweepy.Client(userdata.get('bearer_token'), userdata.get('consumer_key'), userdata.get('consumer_secret'), userdata.get('access_token'), userdata.get('access_token_secret'))

#public_tweets = client.get_home_timeline()

#public_tweets